Example for compressing a tensorlfow 1.x model using freeze_graph and optimize_for_inference_lib from tensorflow.python.tools

In [1]:
import tensorflow as tf
import numpy as np

/Users/danatello/venv/ml-python3-2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/danatello/venv/ml-python3-2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/danatello/venv/ml-python3-2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/danatello/venv/ml-python3-2/lib/python3

In [2]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [3]:
from google.protobuf import text_format

In [4]:
k_in = 10
k_out = 1
m = 1000
scale = .01

In [5]:
training_data = {
    "X": np.random.rand(m, k_in),
    "targets": np.random.rand(m, k_out)
}

In [6]:
test_data = {
    "X": np.random.rand(10, k_in),
    "targets": np.random.rand(10, k_out)
}

In [7]:
def create_model_graph():
    g = tf.Graph()
    with g.as_default():
        X_numeric = tf.placeholder(tf.float32, name="X_numeric")
        W = tf.Variable((np.random.rand(k_in, k_out).astype(np.float32) * 2 - 1) * scale, name="W")
        prediction = tf.matmul(X_numeric, W, name="prediction")
        targets = tf.placeholder(tf.float32, name="targets")
        cost = tf.reduce_sum(tf.squared_difference(prediction, targets))
        optimizer = tf.train.AdamOptimizer()
        grads_and_vars = optimizer.compute_gradients(cost)
        update_vars = optimizer.apply_gradients(grads_and_vars)
        init_operation = tf.global_variables_initializer()
    return g, X_numeric, W, prediction, targets, cost, optimizer, grads_and_vars, update_vars, init_operation

In [8]:
g, X_numeric, W, prediction, targets, cost, optimizer, grads_and_vars, update_vars, init_operation = create_model_graph()

In [9]:
g.as_graph_def()

node {
  name: "X_numeric"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "W/initial_value"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 10
          }
          dim {
            size: 1
          }
        }
        tensor_content: "\031\306A\273\375B\316\273\375\027#<\022{\355;\224\250:;W\3233;\036\217\232\271\233\212\210\273p\213\373;\305\376\027<"
      }
    }
  }
}
node {
  name: "W"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 10
        }
        dim {
          size: 1

In [10]:
[x.name for x in g.as_graph_def().node]

['X_numeric',
 'W/initial_value',
 'W',
 'W/Assign',
 'W/read',
 'prediction',
 'targets',
 'SquaredDifference',
 'Rank',
 'range/start',
 'range/delta',
 'range',
 'Sum',
 'gradients/Shape',
 'gradients/grad_ys_0',
 'gradients/Fill',
 'gradients/Sum_grad/Shape',
 'gradients/Sum_grad/Size',
 'gradients/Sum_grad/add',
 'gradients/Sum_grad/mod',
 'gradients/Sum_grad/Shape_1',
 'gradients/Sum_grad/range/start',
 'gradients/Sum_grad/range/delta',
 'gradients/Sum_grad/range',
 'gradients/Sum_grad/Fill/value',
 'gradients/Sum_grad/Fill',
 'gradients/Sum_grad/DynamicStitch',
 'gradients/Sum_grad/Maximum/y',
 'gradients/Sum_grad/Maximum',
 'gradients/Sum_grad/floordiv',
 'gradients/Sum_grad/Reshape',
 'gradients/Sum_grad/Tile',
 'gradients/SquaredDifference_grad/Shape',
 'gradients/SquaredDifference_grad/Shape_1',
 'gradients/SquaredDifference_grad/BroadcastGradientArgs',
 'gradients/SquaredDifference_grad/scalar',
 'gradients/SquaredDifference_grad/Mul',
 'gradients/SquaredDifference_grad/sub

In [11]:
init_operation.name

'init'

In [12]:
with g.as_default():
    sess = tf.Session()

In [13]:
sess.run(init_operation)

In [14]:
sess.run(W)

array([[-0.00295675],
       [-0.00629461],
       [ 0.00995445],
       [ 0.00724734],
       [ 0.00284818],
       [ 0.00274392],
       [-0.0002948 ],
       [-0.00416691],
       [ 0.00767653],
       [ 0.00927705]], dtype=float32)

In [15]:
sess.run(prediction, feed_dict={X_numeric: test_data['X']})

array([[0.01202406],
       [0.01453314],
       [0.0101665 ],
       [0.01171979],
       [0.00388709],
       [0.02137084],
       [0.01914168],
       [0.01302489],
       [0.0097549 ],
       [0.0129439 ]], dtype=float32)

In [16]:
feed_dict = {
    X_numeric: training_data['X'],
    targets: training_data['targets']
}

In [17]:
for i in range(10):
    sess.run(update_vars, feed_dict=feed_dict)

In [18]:
sess.run(W)

array([[0.00700744],
       [0.00367042],
       [0.01991811],
       [0.0172112 ],
       [0.0128131 ],
       [0.01270794],
       [0.00966839],
       [0.00579655],
       [0.0176411 ],
       [0.01924086]], dtype=float32)

In [19]:
sess.run(prediction, feed_dict={X_numeric: test_data['X']})

array([[0.05540302],
       [0.064004  ],
       [0.05815119],
       [0.05346157],
       [0.05541107],
       [0.06349734],
       [0.06127679],
       [0.06486437],
       [0.03683634],
       [0.05131396]], dtype=float32)

In [20]:
with g.as_default():
    model_saver = tf.train.Saver()

In [21]:
model_saver.save(sess, "./saved_model/saved_model.cpkt")

'./saved_model/saved_model.cpkt'

In [22]:
tf.io.write_graph(g, "./saved_model", "model_graph.pbtxt")

'./saved_model/model_graph.pbtxt'

---

---
load graph

In [23]:
g_pb = tf.Graph()

In [24]:
with open("./saved_model/model_graph.pbtxt") as f_in:
    graph_protobuf = text_format.Parse(f_in.read(), tf.GraphDef())

In [25]:
with g_pb.as_default():
    tf.import_graph_def(graph_def=graph_protobuf, name="")

In [26]:
init_operation_pb = g_pb.get_operation_by_name('init')

In [27]:
g_pb.as_graph_def()

node {
  name: "X_numeric"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "W/initial_value"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 10
          }
          dim {
            size: 1
          }
        }
        tensor_content: "\031\306A\273\375B\316\273\375\027#<\022{\355;\224\250:;W\3233;\036\217\232\271\233\212\210\273p\213\373;\305\376\027<"
      }
    }
  }
}
node {
  name: "W"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 10
        }
        dim {
          size: 1

In [28]:
g_pb

In [29]:
sess_pb = tf.Session(graph=g_pb)

# with g_pb.as_default():
#     model_pb_restorer = tf.train.Saver() # throws error. not sure why
#     sess_pb.run(init_operation_pb)
#     model_pb_restorer.restore(sess_pb, "./saved_model/saved_model.cpkt")

---
load model

In [30]:
g_restored, X_numeric_restored, W_restored, prediction_restored, targets_restored, cost_restored, optimizer_restored, grads_and_vars_restored, update_vars_restored, init_operation_restored = create_model_graph()

In [31]:
g_restored.as_graph_def()

node {
  name: "X_numeric"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "W/initial_value"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 10
          }
          dim {
            size: 1
          }
        }
        tensor_content: "\365,\016\274d\331\007\274u\006\327\272\215\240\026\273\002\021\341\273\327\n\232\273=\217@\272\rU\265\273x\336V\273\270t\013:"
      }
    }
  }
}
node {
  name: "W"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 10
        }
        dim {
         

In [32]:
sess_restored = tf.Session(graph=g_restored)
with g_restored.as_default():
    model_restorer = tf.train.Saver()
    sess_restored.run(init_operation_restored)
    model_restorer.restore(sess_restored, "./saved_model/saved_model.cpkt")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./saved_model/saved_model.cpkt


In [33]:
sess_restored.run(prediction_restored, feed_dict={X_numeric_restored: test_data['X']})

array([[0.05540302],
       [0.064004  ],
       [0.05815119],
       [0.05346157],
       [0.05541107],
       [0.06349734],
       [0.06127679],
       [0.06486437],
       [0.03683634],
       [0.05131396]], dtype=float32)

---

---
freeze graph

In [34]:
targets.name

'targets:0'

In [35]:
X_numeric.name

'X_numeric:0'

In [36]:
input_graph_path = "./saved_model/model_graph.pbtxt"
checkpoint_path = './saved_model/saved_model.cpkt'
input_saver_def_path = ""
input_binary = False
output_node_names = "prediction"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const"
output_frozen_graph_name = './saved_model/frozen_graph.pb'
output_optimized_graph_name = './saved_model/optimized_graph.pb'
clear_devices = True
initializer_nodes = ""

freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, initializer_nodes)

# Optimize for inference

input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["X_numeric"], # an array of the input node(s)
        ["prediction"], # an array of output nodes
        tf.float32.as_datatype_enum)

# Save the optimized graph

f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())

# tf.train.write_graph(output_graph_def, './', output_optimized_graph_name) 

INFO:tensorflow:Restoring parameters from ./saved_model/saved_model.cpkt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 1 variables.
INFO:tensorflow:Converted 1 variables to const ops.
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use tf.gfile.GFile.


---
compress model

In [37]:
def load_frozen_graph(frozen_graph):
    with tf.gfile.GFile(frozen_graph, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
    
    return graph

In [38]:
g_frozen = load_frozen_graph('./saved_model/frozen_graph.pb')

In [39]:
g_frozen.as_graph_def()

node {
  name: "X_numeric"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "W"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 10
          }
          dim {
            size: 1
          }
        }
        tensor_content: "\255\236\345;^\213p;L+\243<~\376\214<\004\356Q<\3704P<0h\036<\376\360\275;\024\204\220<\001\237\235<"
      }
    }
  }
}
node {
  name: "W/read"
  op: "Identity"
  input: "W"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@W"
      }
    }
  }
}
node {
  name: "prediction"
  op: "BatchMatMulV2"
  input: "X_numeric"
  input: "W/read"
  attr {
    key: "T"
    val

In [40]:
prediction = g_frozen.get_tensor_by_name('prediction:0')
X_numeric = g_frozen.get_tensor_by_name('X_numeric:0')

In [41]:
sess_frozen = tf.Session(graph=g_frozen)
sess_frozen.run(prediction, feed_dict={X_numeric: test_data['X']})

array([[0.05540302],
       [0.064004  ],
       [0.05815119],
       [0.05346157],
       [0.05541107],
       [0.06349734],
       [0.06127679],
       [0.06486437],
       [0.03683634],
       [0.05131396]], dtype=float32)